In [229]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

## Cost matrix

In [230]:
cost_matrix = pd.read_csv('costMatrixDistance.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

In [231]:
cost_matrix

,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,1.264695,3.289752,5.079517,4.125355,9.565742,6.672981,3.289752,10.293884,15.466988,13.913255,...,13.804221,10.293884,4.125355,15.535282,10.225981,5.079517,14.802674,3.289752,3.289752,3.899394
ChIJwTC7A55ZwokRPNX9g7ngbaI,3.652491,1.293245,2.402347,3.398711,13.306269,5.361937,1.293245,7.911147,17.828954,13.407891,...,12.554920,7.911147,3.398711,17.193343,14.015943,2.402347,13.395202,1.293245,1.293245,9.058113
ChIJU1XImaNZwokRutunetC8XeE,5.344794,2.462545,1.223754,5.815233,12.162233,2.746551,2.462545,5.210559,16.860678,15.701421,...,14.875472,5.210559,5.815233,17.368442,13.057781,1.223754,11.576710,2.462545,2.462545,8.268289
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.159567,2.779143,5.555309,1.352450,14.815639,8.422352,2.779143,10.851788,17.045178,10.390874,...,9.036164,10.851788,1.352450,14.318078,13.400577,5.555309,16.376429,2.779143,2.779143,9.556437
ChIJ5cPkuBtgwokRn55JgpGqjFA,8.448850,13.636841,10.780676,12.250608,2.086402,4.636728,13.636841,11.720111,10.178064,13.210052,...,17.468174,11.720111,12.266233,13.210052,6.411415,10.780676,15.669211,13.636841,13.636841,3.369102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,5.344794,2.462545,1.223754,5.815233,12.162233,2.746551,2.462545,5.210559,16.860678,15.701421,...,14.875472,5.210559,5.815233,17.368442,13.057781,1.223754,11.576710,2.462545,2.462545,8.268289
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.048655,12.831094,7.774905,8.075834,6.662563,5.516532,12.831094,3.592429,15.030873,19.568436,...,19.109784,3.592429,8.075834,19.568436,8.429137,7.774905,1.876177,12.831094,12.831094,15.773412
ChIJQYmJtx5awokR45_sopkXVNE,3.652491,1.293245,2.402347,3.398711,13.306269,5.361937,1.293245,7.911147,17.828954,13.407891,...,12.554920,7.911147,3.398711,17.193343,14.015943,2.402347,13.395202,1.293245,1.293245,9.058113


In [232]:
# Number of shops to visit
n = cost_matrix.shape[0]

# Calculate M
M = 0
for i in range(n):
    for j in range(n):
        M += cost_matrix.iloc[i,j]

# M cost for the path i->i
for i in range(n):
    cost_matrix.iloc[i,i] = M + 1

In [233]:
cost_matrix

,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,475481.507435,3.289752,5.079517,4.125355,9.565742,6.672981,3.289752,10.293884,15.466988,13.913255,...,13.804221,10.293884,4.125355,15.535282,10.225981,5.079517,14.802674,3.289752,3.289752,3.899394
ChIJwTC7A55ZwokRPNX9g7ngbaI,3.652491,475481.507435,2.402347,3.398711,13.306269,5.361937,1.293245,7.911147,17.828954,13.407891,...,12.554920,7.911147,3.398711,17.193343,14.015943,2.402347,13.395202,1.293245,1.293245,9.058113
ChIJU1XImaNZwokRutunetC8XeE,5.344794,2.462545,475481.507435,5.815233,12.162233,2.746551,2.462545,5.210559,16.860678,15.701421,...,14.875472,5.210559,5.815233,17.368442,13.057781,1.223754,11.576710,2.462545,2.462545,8.268289
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.159567,2.779143,5.555309,475481.507435,14.815639,8.422352,2.779143,10.851788,17.045178,10.390874,...,9.036164,10.851788,1.352450,14.318078,13.400577,5.555309,16.376429,2.779143,2.779143,9.556437
ChIJ5cPkuBtgwokRn55JgpGqjFA,8.448850,13.636841,10.780676,12.250608,475481.507435,4.636728,13.636841,11.720111,10.178064,13.210052,...,17.468174,11.720111,12.266233,13.210052,6.411415,10.780676,15.669211,13.636841,13.636841,3.369102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,5.344794,2.462545,1.223754,5.815233,12.162233,2.746551,2.462545,5.210559,16.860678,15.701421,...,14.875472,5.210559,5.815233,17.368442,13.057781,475481.507435,11.576710,2.462545,2.462545,8.268289
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.048655,12.831094,7.774905,8.075834,6.662563,5.516532,12.831094,3.592429,15.030873,19.568436,...,19.109784,3.592429,8.075834,19.568436,8.429137,7.774905,475481.507435,12.831094,12.831094,15.773412
ChIJQYmJtx5awokR45_sopkXVNE,3.652491,1.293245,2.402347,3.398711,13.306269,5.361937,1.293245,7.911147,17.828954,13.407891,...,12.554920,7.911147,3.398711,17.193343,14.015943,2.402347,13.395202,475481.507435,1.293245,9.058113


## Shop demands

In [234]:
shop_demands = pd.read_csv('2015_shop_locations.csv').set_index('id')['stage','demand(kg)'].astype(float)
shop_demands = shop_demands.loc[shop_demands['stage'] == 1]

## Closest neighbour heuristic

In [235]:
# Start from the storage location, go to the lowest cost shop, then go to the lowest cost shop from there, and so on until all shops are visited

# Create a list of visited shops
visited = ['Depot'] # Start from the storage location

# Create a list of unvisited shops
unvisited = shop_demands.index.to_list()

# Calculate the cost of the route
cost = 0

# Van capacity
van_capacity = 300
remaining_capacity = van_capacity

In [236]:
active_arcs = []  # List to store paths for all trucks
current_arcs = []  # Current path for the active truck

while len(unvisited) > 0:
    current = 'Depot'  # Start from the storage location
    while len(unvisited) >= 0:

        # Get the costs of the unvisited shops
        costs = cost_matrix.loc[current, unvisited]

        # Filter out shops that exceed the remaining capacity
        feasible_shops = [shop for shop in unvisited if shop_demands[shop] <= remaining_capacity]

        if not feasible_shops:
            # If no feasible shops, end the current truck's route and reset
            next_shop = 'Depot'  # Go back to the storage location
            current_arcs.append((current, next_shop))
            active_arcs.append(current_arcs)
            current_arcs = []  # Reset the path for the next truck
            remaining_capacity = van_capacity  # Reset the capacity for the new truck
            break  # Exit the loop to start with a new truck

        # Get the costs of the feasible shops
        feasible_costs = costs[costs.index.astype(str).isin(feasible_shops)]

        # Get the index of the lowest cost shop
        next_shop = feasible_costs.idxmin()

        # Store the arc between current and next_shop
        current_arcs.append((current, next_shop))

        # Add the lowest cost shop to the visited list
        visited.append(next_shop)

        # Remove the lowest cost shop from the unvisited list
        unvisited.remove(next_shop)

        # Add the cost of the route
        cost += feasible_costs.min()

        # Update the remaining capacity
        remaining_capacity -= shop_demands[next_shop]

        # Move to the next shop
        current = next_shop


# Cost and route

In [237]:
# Calculate the cost of the route
print('Cost of the route:', cost)
print('Route:', active_arcs)

Cost of the route: 543.4124980225299
Route: [[('Depot', 'ChIJ0zAa911ZwokRQlmIU8v4Kzs'), ('ChIJ0zAa911ZwokRQlmIU8v4Kzs', 'ChIJm5bO7l1ZwokRHeB-QQwXAdE'), ('ChIJm5bO7l1ZwokRHeB-QQwXAdE', 'ChIJJ8jP9kBZwokRfQdnH4EgVFk'), ('ChIJJ8jP9kBZwokRfQdnH4EgVFk', 'ChIJiWXDYR1cwokRL82F6r4W_bk'), ('ChIJiWXDYR1cwokRL82F6r4W_bk', 'ChIJJYi2sUBZwokRfsq_XXpGvQw'), ('ChIJJYi2sUBZwokRfsq_XXpGvQw', 'ChIJa1YQ25lbwokROdvk22x6vHc'), ('ChIJa1YQ25lbwokROdvk22x6vHc', 'ChIJEdfDyr5bwokRV7yHSUveCJE'), ('ChIJEdfDyr5bwokRV7yHSUveCJE', 'ChIJ5VV4FyJcwokRL6sBtbyxRco'), ('ChIJ5VV4FyJcwokRL6sBtbyxRco', 'ChIJdTjwaaFbwokRn7WZXdx_fR8'), ('ChIJdTjwaaFbwokRn7WZXdx_fR8', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'), ('ChIJG7L-TLVbwokRT36uIrwz2Mo', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'), ('ChIJmZQsMgFbwokRMc5I7kPdQ7w', 'ChIJ6ZQpRmBawokR0kz4HEWfI7U'), ('ChIJ6ZQpRmBawokR0kz4HEWfI7U', 'ChIJIwENLUlawokRrzvvH06NlaE'), ('ChIJIwENLUlawokRrzvvH06NlaE', 'ChIJAX-UKUlawokRv3w8InaZxjE'), ('ChIJAX-UKUlawokRv3w8InaZxjE', 'Depot')], [('Depot', 'ChIJwTC7A55ZwokRPNX9g7ngb